### ¿Que hace este script?
* Calcula para cada AP: trafico.totales.totalConexiones
* Calcula para cada site id: trafico.concurrenciaConexiones

In [10]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [11]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [12]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### Definiendo indice principal con fecha de hoy

In [13]:
indice = parametros.conteo_centros_index
indice_control = parametros.tableros_mintic_control

AttributeError: module 'parametros' has no attribute 'conteo_centros_index'

### Función para generar JSON compatible con ES

In [ ]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud, COD_ISO, entre otros).

In [ ]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud','COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) pd.DataFrame(fields)
except:
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()
    

### leyendo indice cambium-devicedevices

Se lee la información de los dispositivos de red monitoreados por Cambium. En esta lectura no hay referencia de fechas ya que solo hay una ocurrencia por MAC de dispositivo de red.

* site_id es la llave para cruzar con cada centro de conexión.
* mac, IP y name son datos básicos del dispositivo.
* ap_group identifica los dispositivos como INDOOR u OUTDOOR

In [ ]:
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac","ip","ap_group","name","status"]  
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_dev = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    exit()

Se descartan registros con site_id vacios y se limpian los NaN del dataframe

In [ ]:
datos_dev.dropna(subset=['site_id'])
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)
datos_dev.sort_values(['site_id','ap_group'], inplace=True)

Se limpian valores errados de ap group

In [ ]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

SE borran duplicados por MAC

In [ ]:
datos_dev = datos_dev.drop_duplicates('mac')

Se renombran campos según estructura del indice final

In [ ]:
datos_dev = datos_dev.rename(columns={'ap_mac' : 'trafico.macRed','ap_group': 'trafico.apGroup'
                                        , 'ip': 'trafico.IP'
                                        , 'mac' : 'trafico.macRed'
                                        , 'name' : 'trafico.deviceName'
                                        , 'status' : 'trafico.status.macRed'})

### Cambiando nombre de campos y generando location

* Se valida latitud y longitud. Luego se calcula campo location
* Se renombran los campos de semilla

In [ ]:
def get_location(x):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return 'a'
datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla['trafico.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['trafico.location']=datos_semilla['trafico.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

datos_semilla = datos_semilla.rename(columns={'nombre_municipio': 'trafico.nombreMunicipio'
                                              , 'nombre_departamento' : 'trafico.nombreDepartamento'
                                              , 'nombre_centro_pob': 'trafico.localidad'
                                              , 'nombreSede' : 'trafico.nomCentroDigital'
                                              , 'energiadesc' : 'trafico.sistemaEnergia'
                                              , 'COD_ISO' : 'trafico.codISO'
                                              , 'id_Beneficiario' : 'trafico.idBeneficiario'})
datos_semilla.fillna('', inplace=True)

## Se lee información para usuarios recurrencia

Se calcula y agrega al indice principal:
* trafico.concurrenciaConexiones

Se lee el indice recurrencia de conexiones y se compara con el flujo detalle conexiones para el rango dado. Si cruzan, se suma a la cuenta de recurrentes.

In [ ]:
total_docs = 5000000
response = es.search(
    index= parametros.ohmyfi_r_u_index,
    body={
            "_source": ["ultima_conexion", "lugar_cod", "id_usuario"]
    },
    size=total_docs
)
#print(es.info())
elastic_docs = response["hits"]["hits"]
fields = {}
for num, doc in enumerate(elastic_docs):
    source_data = doc["_source"]
    for key, val in source_data.items():
        try:
            fields[key] = np.append(fields[key], val)
        except KeyError:
            fields[key] = np.array([val])

datos_recurrencia = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

Se cuenta la cantidad de usuarios con mas de una conexión

# Concurrencia usuario a indice

In [ ]:
use_these_keys = ['trafico.nomCentroDigital',
                  'trafico.codISO',
                  'trafico.localidad',
                  'trafico.siteID',
                  'trafico.codISO',
                  'trafico.nombreDepartamento',
                  'trafico.sistemaEnergia',
                  'trafico.nombreMunicipio',
                  'trafico.idBeneficiario',
                  'trafico.totales.fechaControl',
                  'trafico.concurrenciaConexiones',
                  'trafico.totales.fecha',
                  'trafico.totales.anyo',
                  'trafico.totales.mes',
                  'trafico.totales.dia',
                  'trafico.totales.hora',
                  'trafico.totales.minuto',
                  'nombreDepartamento',
                    'nombreMunicipio',
                    'idBeneficiario',
                    'fecha',
                    'anyo',
                    'mes',
                    'dia',
                  '@timestamp']
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'Concurrencia-' + document['trafico.siteID'] + '-' +document['trafico.totales.fechaControl']}",
                "_source": filterKeys(document),
            }

In [ ]:
try:
    datos_det_conex_completo.rename(columns={'mac_usuario':'id_usuario'}, inplace=True)
    aux_recurrencia=datos_det_conex_completo[['lugar_cod','id_usuario']].groupby(['id_usuario']).agg(['count']).reset_index()
    aux_recurrencia.columns = aux_recurrencia.columns.droplevel(1)
    aux_recurrencia.rename(columns={'lugar_cod': 'contador'}, inplace=True)
    aux_recurrencia = aux_recurrencia.drop(aux_recurrencia[(aux_recurrencia['contador'] < 2)].index)
    datos_recurrencia = pd.merge(datos_det_conex_completo,  aux_recurrencia, on='id_usuario', how='inner')
    datos_recurrencia = datos_recurrencia[['lugar_cod','fecha_control','id_usuario']].groupby(['lugar_cod','fecha_control']).agg(['count']).reset_index()
    datos_recurrencia.columns = datos_recurrencia.columns.droplevel(1)
    datos_recurrencia.rename(columns={'lugar_cod':'site_id'}, inplace=True)
    datos_recurrencia = pd.merge(datos_semilla,  datos_recurrencia, on='site_id', how='inner')
    datos_recurrencia.rename(columns={'site_id':'trafico.siteID'
                                     ,'id_usuario': 'trafico.concurrenciaConexiones'
                                     ,'fecha_control' : 'trafico.totales.fechaControl'}, inplace=True)
    datos_recurrencia.fillna({'trafico.concurrenciaConexiones':0},inplace=True)
    datos_recurrencia.fillna('', inplace=True)
    datos_recurrencia["trafico.totales.fecha"] = datos_recurrencia["trafico.totales.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    datos_recurrencia["trafico.totales.anyo"] = datos_recurrencia["trafico.totales.fecha"].str[0:4]
    datos_recurrencia["trafico.totales.mes"] = datos_recurrencia["trafico.totales.fecha"].str[5:7]
    datos_recurrencia["trafico.totales.dia"] = datos_recurrencia["trafico.totales.fecha"].str[8:10]
    datos_recurrencia["trafico.totales.hora"] = datos_recurrencia["trafico.totales.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    datos_recurrencia["trafico.totales.minuto"] = datos_recurrencia["trafico.totales.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    datos_recurrencia['nombreDepartamento'] = datos_recurrencia['trafico.nombreDepartamento']
    datos_recurrencia['nombreMunicipio'] = datos_recurrencia['trafico.nombreMunicipio']
    datos_recurrencia['idBeneficiario'] = datos_recurrencia['trafico.idBeneficiario']
    datos_recurrencia['fecha'] = datos_recurrencia['trafico.totales.fecha']
    datos_recurrencia['anyo'] = datos_recurrencia['trafico.totales.anyo']
    datos_recurrencia['mes'] = datos_recurrencia['trafico.totales.mes']
    datos_recurrencia['dia'] = datos_recurrencia['trafico.totales.dia']
    
    datos_recurrencia['@timestamp'] = now.isoformat() 
    salida = helpers.bulk(es, doc_generator(datos_recurrencia))
    print("Fecha: ", now,"- Datos Concurrencia de conexiones en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No hay datos de concurrencia de conexiones para insetar en indice principal:")

### Guardando fecha para control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [ ]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia_trafico_conexiones',
        body = { 'jerarquia_trafico_conexiones': 'trafico_conexiones','trafico.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)